import pandas as pd

In [1]:
import pandas as pd

In [3]:
inga = pd.read_csv(r"C:\Users\Dell\Desktop\Restaurant\Ingas Veggie Heaven.csv", error_bad_lines=False);
chrysaliniotissa = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Panagia Chrysaliniotissa Church.csv', error_bad_lines=False);
centre_visual = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Centre of Visual Arts and Research.csv', error_bad_lines=False);
famagusta = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\Famagusta Gate.csv', error_bad_lines=False);

tripadv_review = pd.concat([inga, chrysaliniotissa, centre_visual, famagusta])
tripadv_review

,r_points,r_text,place
0,40,I ordered through the app. They delivered on t...,Ingas Veggie Heaven
1,30,Our first impression was the warm atmosphere a...,Ingas Veggie Heaven
2,50,We stayed by on the way to a family wedding in...,Ingas Veggie Heaven
3,50,"Cosy Atmosphere, we felt like having dinner at...",Ingas Veggie Heaven
4,30,We had the quiche and nut roast. I Found the n...,Ingas Veggie Heaven
...,...,...,...
90,50,This was originally one of the gates where you...,Famagusta Gate
91,50,Was there a few days ago for my child's concer...,Famagusta Gate
92,40,area for the historical and contextual signifi...,Famagusta Gate
93,50,This gate was the Entrance to the Venisian Wal...,Famagusta Gate


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [7]:
processed_reviews = tripadv_review['r_text'].astype(str).map(preprocess)
processed_reviews

0     [ordered, delivered, time, veggie, lazania, wa...
1     [impression, warm, atmosphere, walked, courtya...
2     [stayed, family, wedding, nearby, church, frie...
3     [cosy, atmosphere, felt, like, having, dinner,...
4     [quiche, roast, roast, bland, unfortunately, c...
                            ...                        
90    [originally, gates, come, walled, city, nicosi...
91    [days, child, concert, enjoyed, acoustic, plac...
92    [area, historical, contextual, significance, c...
93    [gate, entrance, venisian, wallafrom, people, ...
94    [went, event, building, time, seen, huge, like...
Name: r_text, Length: 206, dtype: object

In [8]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]


In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=250,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [15]:
from pprint import pprint

In [16]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.019*"nicosia" + 0.019*"gate" + 0.011*"walls" + 0.010*"visit" + '
  '0.009*"place" + 0.009*"cyprus" + 0.009*"city" + 0.009*"cultural" + '
  '0.008*"venetian" + 0.008*"gates"'),
 (1,
  '0.010*"gate" + 0.009*"nicosia" + 0.009*"cyprus" + 0.008*"interesting" + '
  '0.008*"collection" + 0.007*"place" + 0.006*"visit" + 0.006*"centre" + '
  '0.005*"city" + 0.005*"history"'),
 (2,
  '0.009*"cyprus" + 0.009*"place" + 0.008*"vegetarian" + 0.008*"inside" + '
  '0.007*"restaurant" + 0.007*"visit" + 0.007*"walls" + 0.007*"gate" + '
  '0.007*"food" + 0.006*"nicosia"'),
 (3,
  '0.013*"gate" + 0.013*"history" + 0.012*"cyprus" + 0.008*"city" + '
  '0.007*"walk" + 0.007*"little" + 0.007*"nice" + 0.007*"place" + '
  '0.006*"visit" + 0.006*"museum"'),
 (4,
  '0.028*"food" + 0.019*"vegan" + 0.014*"place" + 0.012*"nicosia" + '
  '0.012*"veggie" + 0.010*"nice" + 0.009*"inga" + 0.009*"good" + 0.008*"worth" '
  '+ 0.007*"tasty"')]


In [17]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.060538 -0.086794       1        1  29.744255
4     -0.095507 -0.035826       2        1  23.413860
2     -0.063830  0.035943       3        1  20.018400
1      0.028626  0.047068       4        1  14.293236
3      0.070173  0.039609       5        1  12.530252, topic_info=             Term       Freq      Total Category  logprob  loglift
88           food  53.000000  53.000000  Default  30.0000  30.0000
153         vegan  37.000000  37.000000  Default  29.0000  29.0000
1129     cultural  15.000000  15.000000  Default  28.0000  28.0000
665         walls  31.000000  31.000000  Default  27.0000  27.0000
959       history  28.000000  28.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
1032       island   2.080058  10.491193   Topic5  -5.8204   0.4589
938         greek   2.050106   9.270723   Topic5  -5.8349   0.5681
951     paintings   2.082011  14.441534   Topic5  -5.8194   0.1403
923    collection   2.072082  14.768049   Topic5  -5.8242   0.1131
391   interesting   2.081810  22.553326   Topic5  -5.8195  -0.3056

[375 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1067      2  0.352017  accessible
1067      4  0.352017  accessible
1669      1  0.292422    actually
1669      5  0.584843    actually
1644      1  0.859794       added
...     ...       ...         ...
147       2  0.494489       worth
147       4  0.044954       worth
147       5  0.089907       worth
821       3  0.904642       yummy
1636      5  0.741079        zero

[626 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 3, 2, 4])